In [1]:
pip install psycopg2-binary
pip install sshtunnel
pip install sqlalchemy

SyntaxError: invalid syntax (<ipython-input-1-ba5ec63f4dda>, line 1)

In [6]:
import psycopg2
import sshtunnel as ssh
import pandas
import yaml

In [4]:
## Save the DB credentials and your username and password in a config file.
## You DO NOT want to write down your password directly on a script.
## Hence you should use a config file. I have chosen to work with .yml file.
## But there are also other options. Make sure you either specify the path 
## of the config file or that the file is at the same location as this .ipynb script

#ymlfile:
#db_creds:
#  dsn: '10.2.221.111..116:8563'
#  user: 'tanzeem.haque'
#  password: 'get_your_own_pw_lol'

#config_file = 'config_mentor.yml'
config_file = 'config_student.yml'

with open(config_file, 'r') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
ssh_address    = cfg['db_creds']['ssh_address']
ssh_username   = cfg['db_creds']['ssh_username']
ssh_password   = cfg['db_creds']['ssh_password']
user           = cfg['db_creds']['user']
password       = cfg['db_creds']['password']


In [8]:
try:

    with ssh.SSHTunnelForwarder(
        (ssh_address, 22),
        #ssh_private_key="</path/to/private/ssh/key>",
        ### in my case, I used a password instead of a private key
        ssh_username=ssh_username,
        ssh_password=ssh_password, 
        remote_bind_address=('localhost', 5432)) as server:

        server.start()
        print ("server connected")

        lp = str(server.local_bind_port)
        params = {
             'database': 'alcdatathon2021',
             'user': user,
             'password': password,
             'host': 'localhost',
             'port': lp
         }

        conn = psycopg2.connect(**params)
        curs = conn.cursor()

        print ("database connected")
        curs.execute("SELECT * from aacomplete_kvalues limit 10;")
        print("The number of parts: ", curs.rowcount)
        row = curs.fetchone()

        while row is not None:
           print(row)
           row = curs.fetchone()

except:
    print ("Connection Failed")

server connected
database connected
The number of parts:  10
('2624022', '93', '13', None, None, None, '93', '13', '4.1', '4.1')
('2624022', None, '15', None, None, None, None, None, '4.1', '4.1')
('2624022', None, '17', None, None, None, None, None, '4.1', '4.1')
('2624022', None, '19', None, None, None, None, None, '4.1', '4.1')
('2624022', None, '21', None, None, None, None, None, '4.1', '4.1')
('2624022', None, '23', None, None, None, None, None, '4.1', '4.1')
('2624022', None, '25', None, None, None, None, None, '4.1', '4.1')
('2624022', None, '27', None, None, None, None, None, '4.1', '4.1')
('2624022', None, '29', None, None, None, None, None, '4.1', '4.1')
('2624022', None, '31', None, None, None, None, None, '4.1', '4.1')


In [31]:
## DBconnect with exasol
C = pyexasol.connect(dsn = dsn, user = user, password = passwd)

In [112]:
## SQL Query. I suggest, you do not make complicated queries here, 
## but you have all liberty and freedom to do that. But I personally
## would suggest do as much as possible in the DB and save the result 
## in a physical table so that all you need to do is to simply "read" it here 
all_sets_sql = 'select * from TEST.TZ_STG_UN_DWH_LEADS_FULL_SETS'

In [113]:
## Export the SQL query result into pandas frame
all_sets_df = C.export_to_pandas(all_sets_sql)

In [114]:
print(all_sets_df)

            set name  PROD_REF_COUNT
0             UN all          456542
1       UN union DWH          627973
2     Total distinct          456567
3            DWH all          171431
4      DWH except UN              25
5  UN and DWH common          171406
6      UN except DWH          285136
